Read in all unformatted files. Each line is a data entry

In [23]:
import pandas as pd
import numpy as np

english_exclusive_pure=[i.strip() for i in list(open("../../data_unformatted/english_exclusive_pure.txt", encoding='utf-8-sig').read().splitlines())]
english_inclusive_pure=[i.strip() for i in list(open("../../data_unformatted/english_inclusive_pure.txt", encoding='utf-8-sig').read().splitlines())]
french_exclusive_pure=[i.strip() for i in list(open("../../data_unformatted/french_exclusive_pure.txt", encoding='utf-8-sig').read().splitlines())]
french_inclusive_pure=[i.strip() for i in list(open("../../data_unformatted/french_inclusive_pure.txt", encoding='utf-8-sig').read().splitlines())]

if len(english_inclusive_pure) != len(english_exclusive_pure):
  raise Exception("uneven enlgish pairs")

if len(french_inclusive_pure) != len(french_exclusive_pure):
  raise Exception("uneven french pairs")

setup for zero-shot

In [24]:
both_exclusive_pure=english_exclusive_pure+french_exclusive_pure
both_inclusive_pure=english_inclusive_pure+french_inclusive_pure

# taskDescriptionOne="classify each of the sentences as inclusive or exclusive."
zero_shot=pd.DataFrame()
both_exclusive_pure

zero_shot["text"] = both_exclusive_pure+both_inclusive_pure
zero_shot["class"] = ['exclusive']*len(both_exclusive_pure)+['inclusive']*len(both_inclusive_pure)
print(zero_shot.head())

                                                text      class
0  African immigrants are only able to seek unqua...  exclusive
1           African Americans are always on welfare.  exclusive
2  He looks Middle Eastern so he is probably here...  exclusive
3  Oriental people built the transcontinental rai...  exclusive
4            Everything is fair, I do not see color.  exclusive


#### setup for few-shot  
- 4 random pairs
- 8 random pairs
- 4 inclusive and 4 exclusive. unrelated
- 8 inclusive and 8 exclusive. unrelated  
  
all are equal english and french.  
no sentence is reused. making datasets unique.

In [25]:
import random as ran

num_total_examples=len(english_inclusive_pure)+len(french_inclusive_pure)
both_example_range = range(num_total_examples)
single_example_range = range(len(english_exclusive_pure))

used_index = []

train_pairs_random_4=pd.DataFrame()
train_pairs_random_8=pd.DataFrame()
train_unrelated_random_8=pd.DataFrame()
train_unrelated_random_16=pd.DataFrame()

train_text_buffer=[]
train_class_buffer=[]

on_english=True
on_inclusive=True

train_pairs_random_4

In [26]:
train_text_buffer.clear()
train_class_buffer.clear()

four_ran_pairs=ran.sample(single_example_range, 4)
print(four_ran_pairs)

for i in four_ran_pairs:
  if on_english:
    train_text_buffer.append(english_inclusive_pure[i])
    train_class_buffer.append(1)
    train_text_buffer.append(english_exclusive_pure[i])
    train_class_buffer.append(0)
  else:
    train_text_buffer.append(french_inclusive_pure[i])
    train_class_buffer.append(1)
    train_text_buffer.append(french_exclusive_pure[i])
    train_class_buffer.append(0)

  on_english=not on_english

train_pairs_random_4["text"]=train_text_buffer
train_pairs_random_4["class"]=train_class_buffer

used_index += four_ran_pairs
train_pairs_random_4

[44, 1, 34, 24]


,text,class
0,Scandinavian culture is known for its cool and...,1
1,Scandinavian culture is cold and unfeeling.,0
2,Les Afro-Américains dans les États dépendent d...,1
3,Les Afro-Américains sont toujours des assistés...,0
4,"African Americans, like all people, have a div...",1
5,"African Americans, with their loud and aggress...",0
6,La culture asiatique englobe une grande variét...,1
7,La culture asiatique est juste une question d'...,0


train_random_pairs_8

In [27]:
train_text_buffer.clear()
train_class_buffer.clear()

eight_ran_pairs=ran.sample(list(set(single_example_range)-set(four_ran_pairs)), 8)
print(eight_ran_pairs)

for i in eight_ran_pairs:
  if on_english:
    train_text_buffer.append(english_inclusive_pure[i])
    train_class_buffer.append(1)
    train_text_buffer.append(english_exclusive_pure[i])
    train_class_buffer.append(0)
  else:
    train_text_buffer.append(french_inclusive_pure[i])
    train_class_buffer.append(1)
    train_text_buffer.append(french_exclusive_pure[i])
    train_class_buffer.append(0)

  on_english=not on_english

train_pairs_random_8["text"]=train_text_buffer
train_pairs_random_8["class"]=train_class_buffer

used_index += eight_ran_pairs
train_pairs_random_8

[30, 66, 59, 10, 32, 28, 27, 39]


,text,class
0,People from the Middle East have different cul...,1
1,People from the Middle East are terrorist.,0
2,Les habitants du Gujarat sont connus pour leur...,1
3,"Les gujaratis sont tous des animaux rustiques,...",0
4,"French culture is known for its fashion, food ...",1
5,French people are all about superficiality and...,0
6,Les Indiens préfèrent peut-être manger avec le...,1
7,Les Indiens n'utilisent pas de couverts.,0
8,"Not all Muslim people are extremists, and it's...",1
9,Muslim people are extremists.,0


4 inclusive and 4 exclusive. unrelated

In [28]:
train_text_buffer.clear()
train_class_buffer.clear()

eight_ran_unrelated=ran.sample(list(set(single_example_range)-set(used_index)), 8)

english_slice=eight_ran_unrelated[0 : (len(eight_ran_unrelated)//2)]
french_slice=eight_ran_unrelated[(len(eight_ran_unrelated)//2) : len(eight_ran_unrelated)]

print(eight_ran_unrelated, english_slice, french_slice)

for i in english_slice:
  if on_inclusive:
    train_text_buffer.append(english_inclusive_pure[i])
    train_class_buffer.append(1)
  else:
    train_text_buffer.append(english_exclusive_pure[i])
    train_class_buffer.append(0)
  on_inclusive = not on_inclusive

for i in french_slice:
  if on_inclusive:
    train_text_buffer.append(french_inclusive_pure[i])
    train_class_buffer.append(1)
  else:
    train_text_buffer.append(french_exclusive_pure[i])
    train_class_buffer.append(0)
  on_inclusive = not on_inclusive

train_unrelated_random_8["text"]=train_text_buffer
train_unrelated_random_8["class"]=train_class_buffer

used_index += eight_ran_unrelated
train_unrelated_random_8

[67, 82, 72, 80, 41, 57, 23, 46] [67, 82, 72, 80] [41, 57, 23, 46]


,text,class
0,People from West Bengal have a rich history an...,1
1,"Africans are just primitives, living in the ju...",0
2,Singapore culture is a melting pot of differen...,1
3,Africa is a continent of poverty and disease.,0
4,Les Espagnols ont une culture et des origines ...,1
5,Les Irlandais ne cultivent et ne boivent que d...,0
6,"La culture arabe est diverse, avec différentes...",1
7,Les Québécois sont arrogants et difficiles à c...,0


8 inclusive and 8 exclusive. unrelated 

In [29]:
train_text_buffer.clear()
train_class_buffer.clear()

sixteen_ran_unrelated=ran.sample(list(set(single_example_range)-set(used_index)), 16)

english_slice=sixteen_ran_unrelated[0 : (len(sixteen_ran_unrelated)//2)]
french_slice=sixteen_ran_unrelated[(len(sixteen_ran_unrelated)//2) : len(sixteen_ran_unrelated)]

print(sixteen_ran_unrelated, english_slice, french_slice)

for i in english_slice:
  if on_inclusive:
    train_text_buffer.append(english_inclusive_pure[i])
    train_class_buffer.append(1)
  else:
    train_text_buffer.append(english_exclusive_pure[i])
    train_class_buffer.append(0)
  on_inclusive = not on_inclusive

for i in french_slice:
  if on_inclusive:
    train_text_buffer.append(french_inclusive_pure[i])
    train_class_buffer.append(1)
  else:
    train_text_buffer.append(french_exclusive_pure[i])
    train_class_buffer.append(0)
  on_inclusive = not on_inclusive

train_unrelated_random_16["text"]=train_text_buffer
train_unrelated_random_16["class"]=train_class_buffer

used_index += sixteen_ran_unrelated
train_unrelated_random_16

[3, 64, 14, 18, 7, 79, 37, 13, 31, 48, 43, 21, 77, 38, 42, 2] [3, 64, 14, 18, 7, 79, 37, 13] [31, 48, 43, 21, 77, 38, 42, 2]


,text,class
0,Many Asian American people contributed to buil...,1
1,Maharashtrians are only interested in money an...,0
2,"Native American cultures are rich and diverse,...",1
3,All Hispanic people are illegal immigrants.,0
4,White people are ethnically similar due to com...,1
5,The only thing that can be found in the Sahara...,0
6,"Native Americans, like all people, have the ab...",1
7,Latino workers are only good for manual labor.,0
8,Les Juifs ont un groupe diversifié d'individus...,1
9,Les Métis ne sont que des personnes de race mi...,0


In [43]:
#ensure all are unique and length = 40.
if (len(used_index)==36 and (len(set(used_index))==len(used_index))):
  print("Length==36 and all entires are unique")
else:
  print("its not good")

used_index_84 = [x+84 for x in used_index]
used_index_168 = [x+84 for x in used_index_84]
used_index_252 = [x+84 for x in used_index_168]
compiled_used_index = used_index+used_index_84+used_index_168+used_index_252

few_shot_test = zero_shot.take(list(set(range(len(zero_shot)))-set(compiled_used_index)))
print(f"zero_shot shape: {zero_shot.shape}\nfew_shot_test shape: {few_shot_test.shape}")

Length==36 and all entires are unique
zero_shot shape: (336, 2)
few_shot_test shape: (192, 2)


*^to maintin complete unrelatedness from train to test for few-shot,  
any language or opposite part of pair is not included in test set.  
as a result, every used_index from any of the four files is removed from every file in the few-shot test set*

##### to csv files.

In [44]:
zero_shot.to_csv("../../data_ready/zero_shot.csv", index=False)
train_pairs_random_4.to_csv("../../data_ready/few_shot/train_pairs_random_4.csv", index=False)
train_pairs_random_8.to_csv("../../data_ready/few_shot/train_pairs_random_8.csv", index=False)
train_unrelated_random_8.to_csv("../../data_ready/few_shot/train_unrelated_random_8.csv", index=False)
train_unrelated_random_16.to_csv("../../data_ready/few_shot/train_unrelated_random_16.csv", index=False)
few_shot_test.to_csv("../../data_ready/few_shot/unused_pairs_for_test_data.csv", index=False)